<a href="https://colab.research.google.com/github/AgustinBustos/no_more_exponential/blob/main/equDiff_nn_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Differential Equations With NN

In [141]:
import torch
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

This is an introduction and first exploration of a recent idea i had while working with the ARIMA model, as always im sure its done somewhere, but maybe differential equations are a bit overvalued (especially in the realm of economics), so my sense is that its going to take some time in order for me to really use the results capabilities. <br/>

We are going to work with the classic equation: $$ \dot{f}=f$$<br/>
giving the result: <br/> $$f=ae^x$$ 

In [142]:
x=np.linspace(-2,2)
y=np.exp(x)
px.line(x=x,y=y).show()

#Functions
Lets define some functions for later

In [143]:
def mse(t1,t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()
  
sig= torch.nn.Sigmoid()
relu = torch.nn.ReLU()
tan = torch.nn.Tanh()

# Main

We are going to have a simple neural network with 1 neuron as input, 100 as hidden layer and 1 as output, so lets define the input xt:

In [144]:
xt=torch.autograd.Variable(torch.linspace(-2,2,50), requires_grad=True)

Now lets define the weights

In [145]:
measure=100
w=torch.rand([1,measure],requires_grad=True) # ojo lo obligo a ser creaciente en la inicialiacion rand
b=torch.rand(measure,requires_grad=True)

w1=torch.rand([measure,1],requires_grad=True)
b1=torch.rand(1,requires_grad=True)

The graph has the exponential function and the network with random weights

In [146]:
def f_hat(x):
  layer1=sig(torch.reshape(x,(-1,1)) @ w+b)  #relu o sig  #b #tan
  result=layer1 @ w1 + b1  #b1
  return result.reshape(-1) 


fig1=px.line(x=xt.reshape(-1).detach().numpy(),y=f_hat(xt).detach().numpy())
fig2=px.line(x=x,y=y)
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()

# Main Function, df_hat/dx

This is the most important function, using the pytorch library its trivial to get the derivative of the network

In [147]:
def f_hat_prime(x):
  f_hat(x).backward(torch.ones_like(f_hat(x)))
  x.grad.zero_()

  f_hat(x).backward(torch.ones_like(f_hat(x)))
  return x.grad



The network and its derivative

In [148]:

px.line(x=list(xt.reshape(-1).detach().numpy())+list(xt.reshape(-1).detach().numpy()),
        y=list(f_hat(xt).detach().numpy())+list(f_hat_prime(xt).detach().numpy())).show()

This is the phase graph, we are going to try to get the points over the 45 degree line (df/dx=f).

In [149]:
fig1=px.scatter(x=list(f_hat_prime(xt).detach().numpy()),y=list(f_hat(xt).detach().numpy()))
fig2=px.line(x=np.linspace(0,7),y=np.linspace(0,7))

fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()

# Main Idea

We have the equation: $$\dot{f}=f$$ We are going to use a network to try and aproximate the solution, so in the end it wont be possible to get an exact equality: $$\dot{\hat{f}}\approx\hat{f}\Longrightarrow\dot{\hat{f}}-\hat{f}=error$$
 And thats the trick, we will have an error of the type: $$\int\left(\dot{\hat{f}}-\hat{f}\right)^2=mse$$
Thats the main idea, analog to linear regression, we can transform a system of equations into an optimization problem and then use neural networks to solve it using gradient descent. <br/> In this case, the real loss is:

```
loss = mse(f_hat(xt), f_hat_prime(xt))
```


Aclaration, maybe it will be necessary to run the following cell multiple times in order to place the points over the 45 degree line, thats because the volatility of training and local minimun problem.


In [151]:
measure=100 
w=torch.rand([1,measure],requires_grad=True)
b=torch.rand(measure,requires_grad=True)

w1=torch.rand([measure,1],requires_grad=True)
b1=torch.rand(1,requires_grad=True)


optimizer = torch.optim.Adam([w,b,w1,b1], lr=0.1) 


for epoch in range(10000):   
    loss = mse(f_hat(xt), f_hat_prime(xt))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
fig1=px.scatter(x=list(f_hat_prime(xt).detach().numpy()),y=list(f_hat(xt).detach().numpy()))
a=f_hat(torch.tensor(0.))[0].item()
fig2=px.line(x=np.linspace(0,a*2.71**2),y=np.linspace(0,a*2.71**2))

fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()


f_hat againts its derivative

In [152]:
px.line(x=list(xt.reshape(-1).detach().numpy())+list(xt.reshape(-1).detach().numpy()),
        y=list(f_hat(xt).detach().numpy())+list(f_hat_prime(xt).detach().numpy())).show()

f_hat against the exponential

In [153]:
fig1=px.line(x=xt.reshape(-1).detach().numpy(),y=f_hat(xt).detach().numpy())
a=f_hat(torch.tensor(0.))[0].item()
fig2=px.line(x=x,y=y*a)
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()

